### Importar librerías

In [8]:
import numpy as np
import pandas as pd
import os
# ---------------------------
# LECTURA DE DATOS
# ---------------------------

# Lista de posibles rutas
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/RDEV.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/RDEV.xlsx',
    'C:/Users/mticllacu/OneDrive - LUZ DEL SUR S.A.A/Archivos de Ronald Quispe Ocaña - ProyectoRyD_V2/Basededatos/RDEV.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):   # verifica si existe
        df = pd.read_excel(path, header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/RDEV.xlsx


### Limpieza de datos

In [9]:
df["SERIE"] = df["SERIE"].astype(str)
df['SERIE'] = df['SERIE'].astype(str).str.replace(" ", "")
# ---------------------------
# LIMPIEZA DE DATOS
# ---------------------------
df = df.iloc[:, 1:]   # quitar primera columna vacía
df['FECHA'] = pd.to_datetime(df['FECHA'], errors="coerce")
df = df.dropna(subset=['FECHA'])
# df["SERIE"] = df["SERIE"].astype(str)
df_full = df.copy()   # copia de detalles originales
df.head()

,SERIE,FECHA,H POS1 R 75°C [mΩ],H POS2 R 75°C [mΩ],H POS3 R 75°C [mΩ],H POS4 R 75°C [mΩ],H POS5 R 75°C [mΩ],H POS6 R 75°C [mΩ],H POS7 R 75°C [mΩ],H POS8 R 75°C [mΩ],...,H POS24 T 75°C [mΩ],H POS25 T 75°C [mΩ],H POS26 T 75°C [mΩ],H POS27 T 75°C [mΩ],X POS1 R 75°C [mΩ],X POS1 S 75°C [mΩ],X POS1 T 75°C [mΩ],Y POS1 R 75°C [mΩ],Y POS1 S 75°C [mΩ],Y POS1 T 75°C [mΩ]
0,59571,2016-10-23,NaN,393.230000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,28.142000,NaN,NaN,15.909,NaN,NaN
1,59571,2006-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59571,1996-04-16,420.991013,406.693652,378.146272,406.622639,421.073862,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,29.068145,NaN,NaN,NaN,NaN,0.0
3,59572,2016-09-18,NaN,424.407154,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,26.451000,NaN,NaN,15.038,NaN,NaN
4,59572,2006-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Lógica

In [10]:
# ---------------------------
# CÁLCULO DE DELTAS
# ---------------------------
res_cols = [c for c in df.columns if c.endswith("[mΩ]")]

for col in res_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    # valor inicial por SERIE en la fecha mínima
    ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
    ref_mapped = df["SERIE"].map(ref)

    # variación porcentual
    
    delta = abs((df[col] - ref_mapped) / ref_mapped) * 100
    delta = delta.replace([np.inf, -np.inf], np.nan)

    # marcar casos inválidos
    mask = df[col].isna() | ref_mapped.isna() | (ref_mapped == 0) | (df[col] == 0)
    delta[mask] = np.nan

    # agregar columna nueva
    df[f"{col}_Delta"] = delta


C:\Users\roquispec\AppData\Local\Temp\ipykernel_14884\3160493815.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppData\Local\Temp\ipykernel_14884\3160493815.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppData\Local\Tem

### Asignación de puntajes

In [11]:
# ---------------------------
# CÁLCULO DE SCORES
# ---------------------------
delta_cols = [c for c in df.columns if c.endswith("_Delta")]

for col in delta_cols:
    puntaje_col = col.replace("_Delta", "_Score")
    df[puntaje_col] = df[col].apply(
        lambda x: np.nan if pd.isna(x) else (1 if x < 5 else 5)
    )
df.head()

C:\Users\roquispec\AppData\Local\Temp\ipykernel_14884\2207295071.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[puntaje_col] = df[col].apply(
C:\Users\roquispec\AppData\Local\Temp\ipykernel_14884\2207295071.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[puntaje_col] = df[col].apply(
C:\Users\roquispec\AppData\Local\Temp\ipykernel_14884\2207295071.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining

,SERIE,FECHA,H POS1 R 75°C [mΩ],H POS2 R 75°C [mΩ],H POS3 R 75°C [mΩ],H POS4 R 75°C [mΩ],H POS5 R 75°C [mΩ],H POS6 R 75°C [mΩ],H POS7 R 75°C [mΩ],H POS8 R 75°C [mΩ],...,H POS24 T 75°C [mΩ]_Score,H POS25 T 75°C [mΩ]_Score,H POS26 T 75°C [mΩ]_Score,H POS27 T 75°C [mΩ]_Score,X POS1 R 75°C [mΩ]_Score,X POS1 S 75°C [mΩ]_Score,X POS1 T 75°C [mΩ]_Score,Y POS1 R 75°C [mΩ]_Score,Y POS1 S 75°C [mΩ]_Score,Y POS1 T 75°C [mΩ]_Score
0,59571,2016-10-23,NaN,393.230000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,59571,2006-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59571,1996-04-16,420.991013,406.693652,378.146272,406.622639,421.073862,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,59572,2016-09-18,NaN,424.407154,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN
4,59572,2006-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# ---------------------------
# ROHM FINAL
# ---------------------------
score_cols = [c for c in df.columns if c.endswith("_Score")]
df["ROHM"] = df[score_cols].max(axis=1)
df_ROHM = df[['SERIE', 'FECHA', 'ROHM']]
df_ROHM.head()

C:\Users\roquispec\AppData\Local\Temp\ipykernel_14884\3243660742.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["ROHM"] = df[score_cols].max(axis=1)


,SERIE,FECHA,ROHM
0,59571,2016-10-23,1.0
1,59571,2006-01-29,NaN
2,59571,1996-04-16,1.0
3,59572,2016-09-18,5.0
4,59572,2006-01-29,NaN


### Extensión de calendario

In [13]:
# ---------------------------
# EXTENSIÓN DEL CALENDARIO DESDE 2025
# ---------------------------
inicio = "2015-01-01"
desde_2025 = f"{pd.Timestamp.today().year}-01-01"
fecha_inicio = pd.Timestamp(inicio)  # en el 2026 cambiar ---****
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")
todas_series = df['SERIE'].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE", "FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

# ---------- Tabla extendida ROHM ----------
ultimos_2024 = df_ROHM[df_ROHM['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024['FECHA'] = fecha_inicio
base_ext = pd.concat([df_ROHM, ultimos_2024], ignore_index=True)
df_extendida = pd.merge(df_calendario, base_ext, on=["SERIE", "FECHA"], how="left")
df_extendida = df_extendida.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# ---------- Tabla extendida detalles ----------
ultimos_2024_det = df_full[df_full['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024_det['FECHA'] = fecha_inicio
base_ext_det = pd.concat([df_full, ultimos_2024_det], ignore_index=True)
df_extendida_detalles = pd.merge(df_calendario, base_ext_det, on=["SERIE", "FECHA"], how="left")
df_extendida_detalles = df_extendida_detalles.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# ---------------------------
# DETALLES + ROHM
# ---------------------------
df_detalles = pd.merge(df_full, df_ROHM, on=["SERIE", "FECHA"], how="left")
df_detalles_ext = pd.merge(df_extendida_detalles, df_extendida, on=["SERIE", "FECHA"], how="left")

# Reordenar columnas: poner ROHM después de FECHA
def reordenar(df_in):
    cols = list(df_in.columns)
    if "ROHM" in cols:
        cols.remove("ROHM")
        idx = cols.index("FECHA") + 1
        cols = cols[:idx] + ["ROHM"] + cols[idx:]
    return df_in[cols]

df_detalles = reordenar(df_detalles)
df_detalles_ext = reordenar(df_detalles_ext)

C:\Users\roquispec\AppData\Local\Temp\ipykernel_14884\4125563871.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida = df_extendida.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_14884\4125563871.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_extendida_detalles = df_extendida_detalles.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykerne

### Creación de funciones

In [14]:

# ---------------------------
# FUNCIONES PARA LLAMAR
# ---------------------------
def get_df_ROHM():
    return df_ROHM

def get_df_extendida_ROHM():
    return df_extendida

def get_df_detalles_ROHM():
    return df_detalles

def get_df_detalles_ext_ROHM():
    return df_detalles_ext

# ---------------------------
# PRINT DE TABLAS
# ---------------------------
print('\n ====== TABLA CON FECHAS ORIGINALES ====== \n')
print(get_df_ROHM(), '\n')
print('\n ====== TABLA CON FECHAS EXTENDIDAS ====== \n')
print(get_df_extendida_ROHM().head(), '\n')
print('\n ====== TABLA DE DETALLES CON FECHAS ORIGINALES ====== \n')
print(get_df_detalles_ROHM().head(), '\n')
print('\n ====== TABLA DE DETALLES CON FECHAS EXTENDIDAS ====== \n')
print(get_df_detalles_ext_ROHM().tail(), '\n')


 ====== TABLA CON FECHAS ORIGINALES ====== 

         SERIE      FECHA  ROHM
0        59571 2016-10-23   1.0
1        59571 2006-01-29   NaN
2        59571 1996-04-16   1.0
3        59572 2016-09-18   5.0
4        59572 2006-01-29   NaN
..         ...        ...   ...
226  PT9003-01 2003-02-26   NaN
227  PT9003-01 2000-04-12   1.0
228    V101038 2024-04-13   5.0
229    V101038 2017-11-02   1.0
230    V101041 2017-10-17   1.0

[231 rows x 3 columns] 


 ====== TABLA CON FECHAS EXTENDIDAS ====== 

     SERIE      FECHA  ROHM
0  123157T 2015-01-01   NaN
1  123157T 2015-01-02   NaN
2  123157T 2015-01-03   NaN
3  123157T 2015-01-04   NaN
4  123157T 2015-01-05   NaN 


 ====== TABLA DE DETALLES CON FECHAS ORIGINALES ====== 

   SERIE      FECHA  ROHM  H POS1 R 75°C [mΩ]  H POS2 R 75°C [mΩ]  \
0  59571 2016-10-23   1.0                 NaN          393.230000   
1  59571 2006-01-29   NaN                 NaN                 NaN   
2  59571 1996-04-16   1.0          420.991013          406.6936